In [ ]:
import matplotlib.image as mpimg
import pylab
import imageio
import cv2
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from PIL import Image
from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
from imutils import build_montages
from imutils import paths
import argparse
import imutils
import scipy.misc
import csv
import glob
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Récupération des Key Frames pour chaque vidéo.

In [ ]:
# extraction de tout les fichier pickels de kframes.
listz=['1','2','2_2','3','4','5','6','7','8','9']
for l in tqdm(listz):
  with ZipFile('/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/kframes/dic'+l+'_kframe.zip', 'r') as zip:
    zip.printdir()
    zip.extractall()

In [ ]:
# Cette la fait sur la globalité de tout les key frames de tout les videos
# Lecture des dictionnaires d'images frames pour les videos
# on va combiner 10 dictionaire dans un seule telle que pour chaque dictionaire on:
  #keys --> l'id de la video
  #values --> les keyframes 
import pickle
listz=['1','2','2_2','3','4','5','6','7','8']

with open('/content/dic8_kframe.pickle', 'rb') as handle:
  dic8= pickle.load(handle)
with open('/content/dic7_kframe.pickle', 'rb') as handle:
  dic7= pickle.load(handle)
with open('/content/dic6_kframe.pickle', 'rb') as handle:
  dic6= pickle.load(handle)
with open('/content/dic9_kframe.pickle', 'rb') as handle:
  dic9= pickle.load(handle
with open('/content/dic5_kframe.pickle', 'rb') as handle:
    dic5= pickle.load(handle)
with open('/content/dic4_kframe.pickle', 'rb') as handle:
  dic4= pickle.load(handle)
with open('/content/dic2_kframe.pickle', 'rb') as handle:
  dic2= pickle.load(handle)
with open('/content/dic1_kframe.pickle', 'rb') as handle:
  dic1= pickle.load(handle)

In [ ]:
## pendant cette étape on a essayer d'extraire tout les dictionaire pickel en même temps,
#  mais le karnel crache à cause l'utilisation massive de mémoire.
list_dic=[]
for l in tqdm(listz):
  with open('/content/dic'+l+'_kframe.pickle', 'rb') as handle:
    dic= pickle.load(handle)
    list_dic.append(dic)
  dic_all.update(d1)


### Traitement des KeyFrames.

* L'objectif de cette étape d'avoir un nombre équitable des Kframes pour chaque vidéo dans ce cas on a  choisi 3  keyframe.

In [ ]:
## la traitement_frames nous permet d'extraire juste trois Kframes pour chaque video.
def traitement_3frames(dic,id):
  if len(dic[id]) == 1:
    dic[id].append(dic[id][0])
    dic[id].append(dic[id][0])
  elif len(dic[id]) == 2:
    dic[id].append(dic[id][1])
  elif len(dic[id]) == 4:
    dic[id]=dic[id][-3:]
  return dic[id][0], dic[id][1], dic[id][2]

def traitement_4frames(dic,id):
  x=int(len(dic[id])/2)
  return dic[id][x], dic[id][x+1], dic[id][x+2]


def traitement_frames(dic):
  list_id=[]
  list_frame0=[]
  list_frame1=[]
  list_frame2=[]
  for id in dic.keys():
    if len(dic[id]) > 4:
      frame0=traitement_4frames(dic,id)[0]
      frame1=traitement_4frames(dic,id)[1]
      frame2=traitement_4frames(dic,id)[2]
    else:
      frame0=traitement_3frames(dic,id)[0]
      frame1=traitement_3frames(dic,id)[1]
      frame2=traitement_3frames(dic,id)[2]
    list_id.append(id)
    list_frame0.append(frame0)
    list_frame1.append(frame1)
    list_frame2.append(frame2)
  return list_id, list_frame0, list_frame1, list_frame2


In [ ]:
# choisir les trois Kframes dans pour chaque video.
list_dic=[dic1,dic2,dic2_2,dic3,dic4,dic5,dic6,dic7,dic8,dic9]
frames0_all=[]
frames1_all=[]
frames2_all=[]
list_id=[]
for k in list_dic:
  list_id.extend(traitement_frames(k)[0])
  frames0_all.extend(traitement_frames(k)[1])
  frames1_all.extend(traitement_frames(k)[2])
  frames2_all.extend(traitement_frames(k)[3])


In [ ]:
df = pd.DataFrame(columns = ['movieId', '1frame','2frame','3frame'])
df['movieId']=list_id
df['1frame']=frames0_all
df['2frame']=frames1_all
df['3frame']=frames2_all
# enrégistrer la dataframe des trois kframes pour chaque id video.
df.to_pickle("./df_3kframe.pkl")

In [ ]:
# lire la dataframe
df_3k = pd.read_pickle("df_3kframe.pkl")

* On obtient une liste des ids de films, une liste de leur frame 0, une liste de leur frame 1, une liste de leur frame 2

###  Fonctions qui permettent l'extraction des visual features

In [ ]:
def sharpness(image):
  kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
  image_sharp = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
  #img_sharp = plt.imshow(image_sharp)
  sharpness = image_sharp.std()
  return sharpness

In [ ]:
def contrast(img):
  img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #plt.imshow(img_grey)
  contrast = img_grey.std()
  return contrast

In [ ]:
def saturation(img):
  img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  #plt.imshow(img_hsv)
  saturation = img_hsv[:, :, 1].mean()
  return saturation


In [ ]:
def brightness(img):
    if len(img.shape) == 3:
        # Colored RGB or BGR (*Do Not* use HSV images with this function)
        # create brightness with euclidean norm
        return np.average(norm(img, axis=2)) / np.sqrt(3)
    else:
        return np.average(img)

In [ ]:
def colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
	rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)

### Construction du Data Frame 

In [ ]:
def compute_features(id,frame0,frame1,frame2):
  features=[]
  f10=sharpness(frame0)
  f11=sharpness(frame1)
  f12=sharpness(frame2)
  f20=contrast(frame0)
  f21=contavecrast(frame1)
  f22=contrast(frame2)
  f30=saturation(frame0)
  f31=saturation(frame1)
  f32=saturation(frame2)
  f40=brightness(frame0)
  f41=brightness(frame1)
  f42=brightness(frame2)
  return id,f10,f11,f12,f20,f21,f22,f30,f31,f32,f40,f41,f42

In [ ]:
def construct_df(list_ids,list_frames0,list_frames1,list_frames2):
  lf0 = []
  for id,f0,f1,f2 in zip(list_ids,list_frames0,list_frames1,list_frames2):
    res = compute_features(id,f0,f1,f2)
    lf0.append(res)
    dfinal = pd.DataFrame(lf0, columns=['id', 'sharpnessframe0','sharpnessframe1','sharpnessframe2',
                                    'contrastframe0', 'contrastframe1','contrastframe2',
                                    'saturationframe0','saturationframe1','saturationframe2',
                                    'brightnessframe0', 'brightnessframe1', 'brightnessframe2'])
  return dfinal

In [ ]:
def compute_featuresframes1(id,frame1):
  features=[]
  f11=sharpness(frame1)
  f21=contrast(frame1)
  f31=saturation(frame1)
  f41=brightness(frame1)
  return id,f11,f21,f31,f41

In [ ]:
def construct_df1(list_ids,list_frames1):
  lf0 = []
  for id,f1 in zip(ids, frames1):
    res = compute_featuresframes1(id,f1)
    lf0.append(res)
    dfinal = pd.DataFrame(lf0, columns=['id','sharpnessframe1','contrastframe1','saturationframe1','brightnessframe1'])
  return dfinal


In [ ]:
# extraction de tout les fichier pickels de kframes.
list_3kf=['df1','df2','df2_3','df4_5','df7_8','df9_6']
path_3k='/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/kframes/'
df1k = pd.read_pickle(path_3k+'df1.pkl')
df2k = pd.read_pickle(path_3k+'df2.pkl')
df3k=pd.read_pickle(path_3k+'df2_3.pkl')
df45k=pd.read_pickle(path_3k+'df4_5.pkl')Extract_Kframes_Iframe_vf.
df78k=pd.read_pickle(path_3k+'df7_8.pkl')
df96k=pd.read_pickle(path_3k+'df9_6.pkl')

# extraction des features (	sharpness,contrast,saturation,brightness) par fonction construct_df.
df1= construct_df(df_3k['movieId'],df_3k['1frame'],df_3k['2frame'],df_3k['3frame'])
df2= construct_df(df2k['movieId'],df2k['1frame'],df2k['2frame'],df2k['3frame'])
df3=construct_df(df3k['movieId'],df3k['1frame'],df3k['2frame'],df3k['3frame'])
df45=construct_df(df45k['movieId'],df45k['1frame'],df45k['2frame'],df45k['3frame'])
df78=construct_df(df78k['movieId'],df78k['1frame'],df78k['2frame'],df78k['3frame'])
df96=construct_df(df96k['movieId'],df96k['1frame'],df96k['2frame'],df96k['3frame'])

frames = [df1, df2, df3, df45, df78, df96]
df_3kf_all = pd.concat(frames)
print(df_3kf_all.shape)
df_3kf_all.head()
df_3kf_all.to_csv('df_3kf_features.csv')

In [ ]:
for l in tavecqdm(list_3kf):
  df_3k = pd.read_pickle(path_3k+l+'.pkl')
print(df_3k .shape)e luminance ou de coul

In [ ]:
with open('dic_kframe_all.pickle', 'wb') as handle:
    pickle.dump(dic_all, handle, protocol=pickle.HIGHEST_PROTOCOL)


### Merge avec le df textuel pour avoir les ratings

* Nous avons les Dataframes avec les visual features pour les 3 frames de chaque vidéo, les variables explicatives seront *

In [ ]:
dframes = pd.read_csv('/Chemins/df_3kf_features.csv')
dframes.head()

,Unnamed: 0,id,sharpnessframe0,sharpnessframe1,sharpnessframe2,contrastframe0,contrastframe1,contrastframe2,saturationframe0,saturationframe1,saturationframe2,brightnessframe0,brightnessframe1,brightnessframe2
0,0,1217.0,0.000000,24.198152,49.518097,0.000000,20.032333,46.482066,0.000000,123.805872,115.495043,0.000000,51.646807,58.780452
1,1,1479.0,29.480947,57.592267,66.545913,21.192217,54.614701,62.853518,132.215162,34.753054,32.865254,45.812622,106.191348,96.619399
2,2,105246.0,83.358822,0.025681,61.488217,82.294308,0.033315,60.256639,66.632231,0.283333,44.012292,92.099638,0.000642,88.614613
3,3,1297.0,44.192303,21.181056,26.265000,41.447294,16.136958,16.589225,47.174363,52.380480,76.325741,127.719641,89.342279,82.455628
4,4,1100.0,50.940310,39.897941,44.513532,28.437360,33.571649,38.492843,203.411908,53.749081,67.228320,64.244570,122.127239,125.014022


* Nous avons le dataframe ddonéé dans le sujet qui contient la variable à expliquer *rating*

In [ ]:
data_original= pd.read_csv('/content/drive/MyDrive/DATA/challengeSIDM2-movies.min30.withtext.csv')
df1= data_original[['movieId','rating']]

In [ ]:
df_merged = df1.merge(dfinal, left_on='movieId', right_on='id')

In [ ]:
dfinal['id']=dfinal['id'].astype(int)
print(dfinal.dtypes)

id                    int64
sharpnessframe1     float64
contrastframe1      float64
saturationframe1    float64
brightnessframe1    float64
dtype: object


In [ ]:
data_final

,movieId,rating,sharpnessframe1,contrastframe1,saturationframe1,brightnessframe1
0,1,3.921240,57.419495,54.635186,93.878275,41.419639
1,10,3.430029,69.772166,64.340808,61.188462,152.417500
2,11,3.667713,2.198145,0.679535,254.946784,2.148668
3,12,2.619766,50.363378,42.368974,0.000000,20.463931
4,13,3.272416,43.005481,32.981148,38.017742,134.694856
...,...,...,...,...,...,...
899,119145,3.632075,69.838819,66.435031,52.443099,60.233606
900,119155,2.678082,57.672359,34.739965,219.435764,71.847997
901,120466,3.346154,51.164541,45.649183,116.836992,43.729914
902,120635,2.824468,48.856673,46.538441,87.957027,25.786646
